In [1]:
from core.simulation_modeller import MultiCaseModeller
from sixgill.definitions import ModelComponents, Parameters

In [2]:
modeller = MultiCaseModeller(
    base_model_path="RDS1_max Flow_Summer.pips",
    excel_path="Water Injection Profile for BAB LP.xlsx",
    sink_profile_sheet="well profile",
    condition_sheet="conditions",
)

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 22816
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:manta.server.manager:Waiting for PIPESIM server to start.


In [3]:
case, condition = "O-Max", "S-HP"

In [4]:
modeller.set_simulation_settings(condition)

INFO:core.simulation_modeller:Set simulation settings for condition: S-HP


In [5]:
modeller.set_parameters_dict(condition)

INFO:core.simulation_modeller:Set parameters for condition: S-HP


In [6]:
modeller.set_sink_data(case, parameter=Parameters.Sink.GASFLOWRATE)

INFO:core.simulation_modeller:Set sink data for case: O-Max


In [7]:
modeller.model.get_values(component=ModelComponents.SINK)

defaultdict(dict,
            {'Sk101': {'Name': 'Sk101',
              'GasFlowRate': 100.0,
              'LiquidFlowRate': None,
              'MassFlowRate': None,
              'Pressure': None,
              'IsActive': True,
              'FlowrateType': 'GasFlowRate'},
             'SINK102': {'Name': 'SINK102',
              'GasFlowRate': 0.0001,
              'LiquidFlowRate': None,
              'MassFlowRate': None,
              'Pressure': None,
              'IsActive': False,
              'FlowrateType': 'GasFlowRate'}})